<a href="https://colab.research.google.com/github/Kiran-Venkatesh/KV/blob/master/car_dekho_form_29.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##################################### DOCUMENTATION BLOCK #######################


################################### LINK BLOCK #################################
import os
from google.cloud import vision
from google.cloud.vision_v1 import types
import re
from fuzzywuzzy import fuzz
import matplotlib.pyplot as plt
import cv2
import traceback
import pandas as pd
from skimage import io
from google.colab.patches import cv2_imshow
import numpy as np
import urllib.request

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
os.listdir('/content/drive/MyDrive')

['redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)',
 'google-vision-key.json',
 'FORM_29_finance_analysis.xlsx',
 'FORM_29_finance_10-03-218-03 PM.csv',
 'FORM_29_finance_10-03-218-03 PM (1).gsheet',
 'FORM_29_finance_10-03-218-03 PM.gsheet',
 'Extraction_Form_29.xlsx',
 'form29_images',
 'form_29_signature']

In [ ]:
dir_path = "/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/"
df = pd.read_csv(os.path.join(dir_path, "FORM_29_finance_10-03-218-03 PM.csv"))
df.head()

,finance_lead_id,name,image_path,city_name,image_abs_path
0,142404,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Ahmedabad,/content/drive/MyDrive/redocumentsloantrackdri...
1,1239,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,New Delhi,/content/drive/MyDrive/redocumentsloantrackdri...
2,170717,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Bangalore,/content/drive/MyDrive/redocumentsloantrackdri...
3,66141,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Surat,/content/drive/MyDrive/redocumentsloantrackdri...
4,187487,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Hyderabad,/content/drive/MyDrive/redocumentsloantrackdri...


In [ ]:
# Downloading images from their file locations
df['image_abs_path'] = 0
image_paths = df['image_path']

form_number = 1
for i, file_path in enumerate(image_paths):
    file_save_path = os.path.join('/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/form29_images/', str(form_number) + '.jpeg')
    urllib.request.urlretrieve(file_path, file_save_path)
    df['image_abs_path'].loc[i] = file_save_path
    form_number += 1




/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [ ]:
df["image_path"][40]

'https://finance-usedgaadi-s3.gaadicdn.com/loan_images/image/132825_1574243921_4.jpg'

In [ ]:
google_vision_key="/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/google-vision-key.json"
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = google_vision_key
vision_client = vision.ImageAnnotatorClient()

def google_ocr(image_path):
    image_data = open(image_path,"rb").read()
    image = types.Image(content = image_data)
    kwargs = {}
    response = vision_client.text_detection(image=image,**kwargs)
    texts = response.text_annotations
    docText=response.full_text_annotation.text
    bboxes = []
    for text in texts:
        bbox = []
        bbox.append(text.description)
        for vertice in text.bounding_poly.vertices:
            bbox.extend([vertice.x,vertice.y])
        bboxes.append(bbox)

    return docText,bboxes

def sig_bbox(bboxes,format):
    try:
        flag = 1
        for i in bboxes[1:]:
            if fuzz.ratio(i[0],'Signature')>90:
                x1 ,y1, x2, y2 = i[1],i[2],i[5],i[6]
                flag = 0
                break

        if flag == 0:
            length = (x2-x1)
            height = (y2-y1)
        
        
        sigx1 = x1 - int(0.2*length)
        sigx2 = x2 + int(1.4*length)
        sigy1 = y1 - int(4*height)
        sigy2 = y1

        return sigx1, sigx2, sigy1, sigy2
    
    except:
        return None

In [ ]:
import re
count=0
for idx, row in df.iterrows():
    image_path = row['image_abs_path']
    img = cv2.imread(image_path)
    
    try:
        bboxes = google_ocr(image_path)
    
        x1,x2,y1,y2 = sig_bbox(bboxes,1)
        cv2.imwrite('/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/form_29_signature/'+row['image_abs_path'].split('/')[-1], img[y1:y2, x1:x2])
    except Exception as e:
        #print(e)
        count += 1
        
print ('{} images does not have signature'.format(count))

200 images does not have signature


In [ ]:
df.shape

(200, 5)

In [ ]:
tf=pd.read_excel("/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/FORM_29_finance_analysis.xlsx")
details_there=df.loc[df.finance_lead_id.isin(tf.loc[(tf["Present Details"] != "Signature of Transferor") & (tf["Form type"] == "Handwritten") & (tf["Language"] != "Hindi")].finance_lead_id.to_list())]


In [ ]:
details_there["Present Details"]=tf.loc[(tf["Present Details"] != "Signature of Transferor") & (tf["Form type"] == "Handwritten") & (tf["Language"] != "Hindi")]["Present Details"]

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
details_there.head()

,finance_lead_id,name,image_path,city_name,image_abs_path,Present Details
5,79638,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Chennai,/content/drive/MyDrive/redocumentsloantrackdri...,"Signature of Transferor,Vehicle No"
13,72512,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,New Delhi,/content/drive/MyDrive/redocumentsloantrackdri...,"Make,Vehicle No,Signature of Transferor,Regist..."
15,112000,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Pune,/content/drive/MyDrive/redocumentsloantrackdri...,"Transferee name,Vehicle No,Signature of Trasnf..."
16,142572,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Chennai,/content/drive/MyDrive/redocumentsloantrackdri...,"Vehicle No,Make,Chassis No,Engine No,Signature..."
18,170253,Form 29,https://finance-usedgaadi-s3.gaadicdn.com/loan...,Pune,/content/drive/MyDrive/redocumentsloantrackdri...,"Signature of Transferor,Transferee name"


In [ ]:
import spacy
import spacy.displacy as displacy

In [ ]:
!python3 -m spacy download en_core_web_md
!python3 -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')
     |████████████████████████████████| 12.0MB 4.7MB/s 
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp37-none-any.whl size=12011741 sha256=778bf6aaeac5439f40cae88c7c1d5f11c5fafa643df8703c2922f78fe85127c1
  Stored in directory: /tmp/pip-ephem-wheel-cache-cecha_zx/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
  Found existing installation: en-core-web-sm 2.2.0
    Uninstalling en-core-web-sm-2.2.0:
      Successfully uninstalled en-core-web-sm-2.2.0
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [ ]:
import re
import string
table = str.maketrans('', '', string.punctuation)
dic={"ID":[],"Image_name":[],"Present_Details":[],"regex_Vehicle_No":[],"regex_Chassis_No":[],"regex_Engine_No":[],"regex_Date":[],"regex_Address":[],"fuzz_Address":[],"fuzz_Vehicle_No":[],"fuzz_Make":[],"fuzz_Chassis_No":[],"fuzz_Engine_No":[],"fuzz_Transferee name":[],"fuzz_Name of Transferee's parent or spouse":[],"NER_DATE":[],"NER_PERSON":[],"NER_ORG":[],"NER_FAC":[]}   

nlp=spacy.load("en_core_web_md")
for idx, row in details_there.iterrows():
  image_path = row['image_abs_path']
  img = cv2.imread(image_path)

  try:
      doctext,bboxes = google_ocr(image_path)
      text=[]
      for txt in bboxes[1:]:
        #print("Before:",txt[0])
        txt[0] = txt[0].translate(table)
        #txt[0] = txt[0].replace(' ', '')
        txt[0] = txt[0].replace('.', ' ')
        txt[0] = txt[0].replace('-', ' ')
        #print("After:",txt[0])
        text.append(txt[0])
      ss=' '.join(text)
      doc=nlp(ss)
      print(idx)
      print(row['Present Details'])
      print(row['image_abs_path'].split('/')[-1])
      
      print(ss)
      print("\n")
      if row["Present Details"] == "Signature of Transferor,Vehicle No":
        dic["NER_DATE"].append("")
        dic["NER_PERSON"].append("")
        dic["NER_ORG"].append("")
        dic["NER_FAC"].append("")
        continue
        

      for ent in doc.ents:
        if ent.label_ in ["PERSON","DATE","ORG","FAC"]:
          dic["NER_"+ent.label_].append(ent.text)
          print(ent.text,"  ---  ",ent.label_)

      
      print("\n")
      print("\n")
      # doctext = doctext.translate(table)
      # doctext = doctext.replace('.', '')
      # doctext = doctext.replace('-', '')
      # doc=nlp(doctext)
      # print(doctext) 
      # print("\n")
      # for ent in doc.ents:
      #   print(ent.text,"  ---  ",ent.label_)

  except Exception as e:
      print(e)
      


OSError: ignored

In [ ]:
displacy.serve(doc,style='ent')

In [ ]:
import string

import re
L=[]
#idx=130
#row=details_there
#dic={"ID":[],"Image_name":[],"Present_Details":[],"regex_Vehicle_No":[],"regex_Chassis_No":[],"regex_Engine_No":[],"regex_Date":[],"regex_Address":[],"fuzz_Address":[],"fuzz_Vehicle_No":[],"fuzz_Make":[],"fuzz_Chassis_No":[],"fuzz_Engine_No":[],"fuzz_Transferee name":[],"fuzz_Name of Transferee's parent or spouse":[]}   

try_count = 0
table = str.maketrans('', '', string.punctuation)
for idx, row in details_there.iterrows():
  l=[]
  dic["ID"].append(idx)
  dic["Image_name"].append(row['image_abs_path'].split('/')[-1])
  dic["Present_Details"].append(row['Present Details'])
  
  l.append(idx)
  l.append(row['image_abs_path'].split('/')[-1])
  l.append(row['Present Details'])
  image_path = row['image_abs_path']#[idx-1]
  img = cv2.imread(image_path)

  try:
      try_count += 1
      print(idx)
      print(row['Present Details'])
      print(row['image_abs_path'].split('/')[-1])
      text,bboxes = google_ocr(image_path)
      for i in bboxes[1:]:
        i[0] = i[0].translate(table)
        #i[0]=i[0].strip()
        #i[0] = i[0].replace(' ', '')
        i[0] = i[0].replace('.', '')
        i[0] = i[0].replace('-', '')

        vehno=re.search("^[A-Z|a-z]{2}\s?[0-9]{1,2}\s?[A-Z|a-z]{0,3}\s?[0-9]{4}$",i[0])
        chassisno=re.search("(?=.*\d|[A-Z])(?=.*[A-Z])[A-Z0-9]{17}",i[0])
        engineno=re.search("^[A-Z|a-z]{3}[0-9]{6}$",i[0])
        date=re.search("^(0?[1-9]|[12][0-9]|3[01])[\/\-](0?[1-9]|1[012])[\/\-]\d{4}$",i[0])
        addr=re.search("^[1-9]{1}[0-9]{2}\\s{0, 1}[0-9]{3}$",i[0])
        if vehno is not None:
          print(idx,"Vehicle No:-",vehno.group(0))
          l.append(vehno.group(0))
          dic["regex_Vehicle_No"].append(vehno.group(0))

        

        if chassisno is not None:
          print(idx,"Chassis No:-",chassisno.group(0))
          l.append(chassisno.group(0))
          dic["regex_Chassis_No"].append(chassisno.group(0))
        

        if engineno is not None:
          print(idx,"Engine No:-",engineno.group(0))
          l.append(engineno.group(0))
          dic["regex_Engine_No"].append(engineno.group(0))
        

        if date is not None:
          print(idx,"Date:-",date.group(0))
          l.append(date.group(0))
          dic["regex_Date"].append(date.group(0))
        

        if addr is not None:
          print(idx,"Address:-",addr.group(0))
          l.append(addr.group(0))
          dic["regex_Address"].append(addr.group(0))
         
        

      for txt in bboxes[0][0].split("\n"):
         if fuzz.token_set_ratio("resident at",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Address"].append(txt)
          
        

         if fuzz.token_set_ratio("vehicle no.",txt)>90 or fuzz.token_set_ratio("have on the",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Vehicle_No"].append(txt)
          
        

         if fuzz.token_set_ratio("make",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Make"].append(txt)
          

         if fuzz.token_set_ratio("chassis no",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Chassis_No"].append(txt)
          

         if fuzz.token_set_ratio("engine no",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Engine_No"].append(txt)
          

         if fuzz.token_set_ratio("shrismt",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Transferee name"].append(txt)
          

         if fuzz.token_set_ratio("son",txt)>90:
           print(txt)
           l.append(txt)
           dic["fuzz_Name of Transferee's parent or spouse"].append(txt)
          
      L.append(l)
         
      print("\n")
     
      #print(text)
     #print("\n")
      #print(bboxes[0][0])
      #print("\n")
  except Exception as e:
      print(e)
        
#extract=pd.DataFrame(L,columns=["ID","Image_name","Present_Details","regex_Vehicle_No","regex_Chassis_No","regex_Engine_No","regex_Date","regex_Address","fuzz_Address","fuzz_Vehicle_No","fuzz_Make","fuzz_Chassis_No","fuzz_Engine_No","fuzz_Transferee name","fuzz_Name of Transferee's parent or spouse"])
#extract=pd.DataFrame(dic)

5
Signature of Transferor,Vehicle No
6.jpeg
have on the
My / Our Vehicle No. IN O4AE 13 08
Make
Chassis No.
Engine No.
(Name) Son/Wife


13
Make,Vehicle No,Signature of Transferor,Registered Authority Signature
14.jpeg
13 Chassis No:- U65999MH2009FTC18
have on the
sold and delivered my/our vehicle No.
Make Vento 16 Chassis No.
Make Vento 16 Chassis No.
Engine No.
(Son / Wife / daughter of
No.
No.
No.


15
Transferee name,Vehicle No,Signature of Trasnferor
16.jpeg
resident
have on the HOIBB993I
Venicle No Make
Chassis No.
Engine No.
to shriSmt.
(Name) son/wife/daughter of
resident
have on the MHOIBR 993)
Vehicle No Make
Vehicle No Make
Chassis No.
Engine No.
(Name) son/wife/daughter of.


16
Vehicle No,Make,Chassis No,Engine No,Signature of Transferor
17.jpeg
16 Chassis No:- MDHHSNAWSG4028792
16 Engine No:- NoE034306
resident
have on the
:Chassis No.
. Make AI.SSAN TERRANO XE I'5 DIESESPS
.... Engine No..E034306
(Son/Wife) daughter of
Shri/Smt
No. .
No. .
No. .
The


18
Signature of Tra

In [ ]:
print ('Full details or partial details were extracted for {} forms our of {} forms'.format(try_count, df.shape[0]))

Full details or partial details were extracted for 71 forms our of 200 forms


In [ ]:
extract = pd.DataFrame.from_dict(dic, orient='index')
extract = extract.transpose()

In [ ]:
extract.to_excel("/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/Extraction_Form_29.xlsx")

In [ ]:

import re 
count = 0
for idx, row in df.iterrows():
#idx=32
#row=df  
  image_path = row['image_abs_path']#[idx-1]
  img = cv2.imread(image_path)

  try:
    bboxes = google_ocr(image_path)
    for i in bboxes[1:]:
      vehno=re.search("^[.-]*[A-Z|a-z]{2}\s?[0-9]{1,2}\s?[A-Z|a-z]{0,3}\s?[0-9]{4}[.-]*$",i[0])
      chassisno=re.search("[.-]*(?=.*\d|[A-Z])(?=.*[A-Z])[A-Z0-9]{17}[.-]*",i[0])
      engineno=re.search("^[.-]*[A-Z|a-z]{3}[0-9]{6}[.-]*$",i[0])
      date=re.search("^(0?[1-9]|[12][0-9]|3[01])[\/\-](0?[1-9]|1[012])[\/\-]\d{4}$",i[0])
      if vehno is not None:
        print(idx,"Vehicle No",vehno.group(0))
      if chassisno is not None:
        print(idx,"Chassis No",chassisno.group(0))
      if engineno is not None:
        print(idx,"Engine No",engineno.group(0))
      if date is not None:
        print(idx,"Date",date.group(0))
        
    text=bboxes[0][0].split("\n")
    print(text)
    for j in text:
      vehno=re.search("[.-]*\d?\s?[A-Z|a-z]{2}\s?[0-9]{1,2}\s?[A-Z|a-z]{0,3}\s?[0-9]{4}[.-]*\d?\s?",j)
      chassisno=re.search("[.-]*\d?\s?(?=.*\d|[A-Z])(?=.*[A-Z])[A-Z0-9]{17}[.-]*\d?\s?",j)
      Chase=re.search("chassis",j,flags=re.IGNORECASE)
      make=re.search("make",j,flags=re.IGNORECASE)
      engine=re.search("engine",j,flags=re.IGNORECASE)
      engineno=re.search("[.-]*\d?\s?[A-Z|a-z]{3}[0-9]{6}[.-]*\d?\s?",j)
      tferor=re.search("Transferee resides",j,flags=re.IGNORECASE)
      address=re.search("resident of.",j,flags=re.IGNORECASE)
      feree=re.search("shri",j,flags=re.IGNORECASE)
      elder=re.search("son",j,flags=re.IGNORECASE)
      if tferor is not None:
        print(idx,"Transferor Name",text[text.index(j)+1])
  
      if address is not None:
        print(idx,"Address",text[text.index(j)-1],j,text[text.index(j)+1])
  
      if feree is not None:
        print(idx,"Transferee Name",text[text.index(j)-1],text[text.index(j)+1])
  
      if elder is not None:
        print(idx,"Elder",text[text.index(j)-1],j,text[text.index(j)+1])
  
      if vehno is not None:
        print(idx,"Vehicle No",vehno.group(0))
      if chassisno is not None:
        print(idx,"Chassis No",chassisno.group(0))
      if engineno is not None:
        print(idx,"Engine No",engineno.group(0))
      if Chase is not None:
        print(idx,"Chassis No",text[text.index(j)-1],j,text[text.index(j)+1])
      if make is not None:
        print(idx,"Make",text[text.index(j)-1],j,text[text.index(j)+1])
      if engine is not None:
        print(idx,"Engine No",text[text.index(j)-1],j,text[text.index(j)+1])
  
    # for j in text:
    #   names=re.findall("^([A-Za-z]+\s?)+$",j)
    #   if len(names)>0:    
    #     print(idx,"Names",names)
    #     print(text)

  except Exception as e:
    print(e)
    pass
    

expected string or bytes-like object
list index out of range
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
list index out of range
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-like object
expected string or bytes-li

In [ ]:
df.to_csv('/content/drive/MyDrive/redocumentsloantrackdrivinglicenserccopy.zip (Unzipped Files)/FORM_29_finance_10-03-218-03 PM.csv', index=False)